# Dataset Statistics for Perception Package Projects
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [Perception package](https://github.com/Unity-Technologies/com.unity.perception) and visualize dataset statistics. It includes statistics and visualizations of the outputs built into the Perception package and should give a good idea of how to use datasetinsights to visualize custom annotations and metrics.

## Setup dataset
If the dataset was generated locally, point `data_root` below to the path of the dataset. The `GUID` folder suffix should be changed accordingly.   

In [ ]:
data_root = "/data/zMvA31N"

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `access_token` in the following cell. The `access_token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [ ]:
from datasetinsights.io.downloader import UnitySimulationDownloader

#run execution id:
run_execution_id = "zMvA31N"
#access_token:
access_token = "uXpdOn9rK8YxOb9sRpAayXCL2oHg_3u14fk_PRlqgsE001f"
#annotation definition id:
annotation_definition_id = "6716c783-1c0e-44ae-b1b5-7f068454b66e"
#unity project id
project_id = "63709827-ed41-46ba-844c-4d4b2eac14fb"
source_uri = f"usim://{project_id}/{run_execution_id}"

downloader = UnitySimulationDownloader(access_token=access_token)

Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation.


In [ ]:
downloader.download(source_uri=source_uri, output=data_root)

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.data.simulation`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [ ]:
from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions
ann_def = AnnotationDefinitions(data_root)
ann_def.table

In [ ]:
metric_def = MetricDefinitions(data_root)
metric_def.table

## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.data.datasets.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [ ]:
from datasetinsights.stats.statistics import RenderedObjectInfo
import datasetinsights.datasets.unity_perception.metrics as metrics
from datasetinsights.datasets.unity_perception.exceptions import DefinitionIDError
from datasetinsights.stats import bar_plot, histogram_plot, rotation_plot

max_samples = 10000          # maximum number of samples points used in histogram plots

rendered_object_info_definition_id = "5ba92024-b3b7-41a7-9d3f-c03a6a8ddd01"
roinfo = None
try:
    roinfo = RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

### Descriptive Statistics

In [ ]:
if roinfo is not None:
    print(roinfo.num_captures())
    roinfo.raw_table.head(3)

### Total Object Count

In [ ]:
if roinfo is not None:
    total_count = roinfo.total_counts()
    display(total_count)
    
    display(bar_plot(
        total_count, 
        x="label_id", 
        y="count", 
        x_title="Label Name",
        y_title="Count",
        title="Total Object Count in Dataset",
        hover_name="label_name"
    ))

### Per Capture Object Count

In [ ]:
if roinfo is not None:
    per_capture_count = roinfo.per_capture_counts()
    display(per_capture_count.head(10))

In [ ]:
if roinfo is not None:
    display(histogram_plot(
        per_capture_count, 
        x="count",  
        x_title="Object Counts Per Capture",
        y_title="Frequency",
        title="Distribution of Object Counts Per Capture",
        max_samples=max_samples
    ))

### Object Visible Pixels

In [ ]:
if roinfo is not None:
    display(histogram_plot(
        roinfo.raw_table, 
        x="visible_pixels",  
        x_title="Visible Pixels Per Object",
        y_title="Frequency",
        title="Distribution of Visible Pixels Per Object",
        max_samples=max_samples
    ))

## Annotation Visualization
In the following sections we show how to load annotations from the Captures object and visualize them. Similar code can be used to consume annotations for model training or visualize and train on custom annotations.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [ ]:
downloader.download(source_uri=source_uri, output=data_root, include_binary=True)

### Load captures

In [ ]:
from datasetinsights.datasets.unity_perception.captures import Captures
cap = Captures(data_root)
cap.captures.head(3)

### Bounding Boxes
In this section we render 2d bounding boxes on top of the captured images.

In [ ]:
from pathlib import Path
def cleanup(catalog):
    catalog = remove_captures_with_missing_files(data_root, catalog)
    catalog = remove_captures_without_bboxes(catalog)
    return catalog

def remove_captures_without_bboxes(catalog):
    keep_mask = catalog["annotation.values"].apply(len) > 0
    return catalog[keep_mask]

def remove_captures_with_missing_files(root, catalog):
    def exists(capture_file):
        path = Path(root) / capture_file
        return path.exists()
    keep_mask = catalog.filename.apply(exists)
    return catalog[keep_mask]

def capture_df(def_id):
    captures = Captures(data_root)
    catalog = captures.filter(bounding_box_definition_id)
    catalog=cleanup(catalog)
    return catalog

def label_mappings_dict(def_id):
    annotation_def = AnnotationDefinitions(data_root)
    init_definition = annotation_def.get_definition(bounding_box_definition_id)
    label_mappings = {
        m["label_id"]: m["label_name"] for m in init_definition["spec"]
    }
    return label_mappings

In [ ]:
import os

from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image

from datasetinsights.stats.visualization.plots import plot_bboxes
from datasetinsights.datasets.synthetic import read_bounding_box_2d

bounding_box_definition_id = "c31620e3-55ff-4af6-ae86-884aa0daa9b2"

try:
    catalog= capture_df(bounding_box_definition_id)
    label_mappings=label_mappings_dict(bounding_box_definition_id)
except DefinitionIDError:
    print("No bounding boxes found")
    
def draw_bounding_boxes(index):
    cap = catalog.iloc[index]
    capture_file = cap.filename
    ann = cap["annotation.values"]
    capture = Image.open(os.path.join(data_root, capture_file))
    image = capture.convert("RGB")  # Remove alpha channel
    bboxes = read_bounding_box_2d(ann, label_mappings)
    return plot_bboxes(image, bboxes, label_mappings)

In [ ]:
from ipywidgets import interact

# pick an index and visualize
interact(draw_bounding_boxes, index=list(range(len(catalog))))

In [ ]:
from thea.datasets import Dataset
data_path = "/data/groceries"
raw_real_images = Dataset.create("GroceriesReal", data_path=data_path, split="train")

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

def draw_bounding_boxes(index):
    cap = catalog.iloc[index]
    capture_file = cap.filename
    ann = cap["annotation.values"]
    capture = Image.open(os.path.join(data_root, capture_file))
    image = capture.convert("RGB")  # Remove alpha channel
    bboxes = read_bounding_box_2d(ann, label_mappings)
    return plot_bboxes(image, bboxes, label_mappings)

In [ ]:
def get_downloaded_images(catalog):
    images = []
    for index in range(len(catalog[:1000])):
        cap = catalog.iloc[index]
        capture_file = cap.filename
        capture = Image.open(os.path.join(data_root, capture_file))
        image = capture.convert("L")
        npix = image.size[1]
        image = image.resize((npix, npix))
        image = np.asarray(image)
        images.append(image)
    images = np.dstack(images)
    images = np.rollaxis(images, -1)
    return images

def get_real_images_numpy(real_images):
    images = []
    for i in range(len(real_images)):
        image = real_images[i][0]
        image = image.convert("L")
        npix = image.size[1]
        image = image.resize((npix, npix))
        image = np.asarray(image)
        images.append(image)
    images = np.dstack(images)
    images = np.rollaxis(images, -1)
    return images

In [ ]:
def get_image_power_spectrum(image):
    ''' Get power spectrum for one image
    
    Args:
        image: 2d numpy array (shape H, W)
    
    Returns:
        fourier_amplitudes: 2d numpy array (shape H, W)
        fourier_image: 2d numpy array (shape H, W)
        Abins: 1d numpy array, azimuthal averaged power spectrum density (shape: (H * W / 2 + 1,))
    '''
    npix = image.shape[0]
    # Fourier transform of two dimensional image data array. (960, 960)
    fourier_image = np.fft.fftn(image) # fft2
    # we only require the square of the amplitudes to compute the variances. (960, 960)
    fourier_amplitudes = np.abs(fourier_image)**2 
    fourier_image = np.fft.fftshift(fourier_image)
    
    # To bin the results found above in k space, we need to know what the layout of
    # the return value of numpy.fft.fftn is.
    # # This will automatically return a one dimensional array containing the wave vectors 
    # for the numpy.fft.fftn call, in the correct order. (960,)
    kfreq = np.fft.fftfreq(npix) * npix
    # Convert this to a two dimensional array matching the layout of the two dimensional Fourier image. [(960, 960), (960, 960)]
    kfreq2D = np.meshgrid(kfreq, kfreq)
    # we are not really interested in the actual wave vectors, but rather in their norm
    knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)
    # we no longer need the wave vector norms or Fourier image to be laid out as a two dimensional array,
    # so we will flatten them
    knrm = knrm.flatten()
    fourier_amplitudes = fourier_amplitudes.flatten()
    # To bin the amplitudes in k space, we need to set up wave number bins. We will create integer k value bins
    kbins = np.arange(0.5, npix//2+1, 1.)
    # The kbin array will contain the start and end points of all bins; the corresponding k values
    # are the midpoints of these bins
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    # Compute the average Fourier amplitude (squared) in each bin
    Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                         statistic = "mean",
                                         bins = kbins)
    # We want the total variance within each bin. Right now, we only have the average power.
    # To get the total power, we need to multiply with the volume in each bin 
    Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)
    return fourier_image, kvals, Abins


In [ ]:
# The analysis will only work for a square image
def get_average_fourier_amplitude(images):
    ''' Average fourier amplitude across images
    
    Args:
        images: 3d numpy array (num of images, H, W)
    
    Returns:
        kvals:
        average_amplitude: 
    '''
    npix = images.shape[1]
    average_amplitude = np.zeros(npix // 2)
    total_fourier_image = np.zeros((npix, npix), dtype='complex128')
    # image size 960 * 960
    for image in images:   
        fourier_image, kvals, Abins = get_image_power_spectrum(image)
        # Sum all the amplitude at each bin
        average_amplitude += Abins
        # shum all image
        total_fourier_image += fourier_image
    n = images.shape[0]
    # take average
    average_amplitude /= n
    averaged_fourier_image = total_fourier_image / n
    return kvals, averaged_fourier_image, average_amplitude

In [ ]:
import plotly.express as px

def plot_fourier_image(data1=None, title1 = "", data2=None, title2=""):
    ''' Plot averaged fourier images of two datasets

    Args:
        data1: numpy.ndarray (shape H, W)
        title1: string, title of the first plot
        data2: numpy.ndarray (shape H, W)
        title2: string, title of the second plot

    Returns:
    '''
    image1 = np.log(np.abs(data1) ** 2)
    image2 = np.log(np.abs(data2) ** 2)
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(1,2,1)
    ax.imshow(image1)
    ax.set_title(title1)
    ax1 = fig.add_subplot(1,2,2)
    ax1.imshow(image2)
    ax1.set_title(title2)

def plot_images(image1=None, title1="", image2=None, title2=""):
    ''' Plot averaged fourier images of two datasets
    
    Args:
        data1: PIL image (shape H, W)
        title1: string, title of the first plot
        data2: PIL image (shape H, W)
        title2: string, title of the second plot
    
    Returns:
    '''
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(1,2,1)
    ax.imshow(image1)
    plt.axis('off')
    ax.set_title(title1)
    ax1 = fig.add_subplot(1,2,2)
    ax1.imshow(image2)
    ax1.set_title(title2)
    plt.axis('off')

In [ ]:
synth_images = get_downloaded_images(catalog)

In [ ]:
real_images = get_real_images_numpy(raw_real_images)

In [ ]:
kvals_synth, averaged_fourier_image_synth, average_amplitude_synth = get_average_fourier_amplitude(synth_images)

In [ ]:
kvals_real, averaged_fourier_image_real, average_amplitude_real = get_average_fourier_amplitude(real_images)

In [ ]:
with open('average_amplitude_real.txt', 'w') as filehandle:
    for listitem in average_amplitude_real:
        filehandle.write('%s\n' % listitem)

In [ ]:
with open('kvals_real.txt', 'w') as filehandle:
    for listitem in kvals_real:
        filehandle.write('%s\n' % listitem)

In [ ]:
plot_fourier_image(data1=averaged_fourier_image_synth, title1 = "Synthetic data", data2=averaged_fourier_image_real, title2="Real-world data")


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=kvals_synth,
    y=average_amplitude_synth,
    mode='lines+markers',
    line=dict(dash='dot'),
    name="synth"
))
fig.add_trace(go.Scatter(
    x=kvals_real,
    y=average_amplitude_real,
    mode='lines+markers',
    line=dict(dash='dot'),
    name="real-world"
))

fig.update_layout(
    title="Azimuthal averaged power spectrum density",
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="k",
    yaxis_title="P(k)"
)
fig.show()

In [ ]:
def convert_numpy_image(image):
    image = image.convert("L")
    npix = image.size[1]
    image = image.resize((npix, npix))
    image = np.asarray(image)
    return image

In [ ]:
index = 100
original_image = raw_real_images[index][0]
original_numpy_image = convert_numpy_image(original_image)

fourier_image, kvals, Abins = get_image_power_spectrum(original_numpy_image)

In [ ]:
plt.imshow(np.log(np.abs(fourier_image) ** 2))

In [ ]:
plt.imshow(original_image)
plt.axis("off")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=kvals,
    y=Abins,
    mode='lines+markers',
    name='original'
))
  
fig.update_layout(
    title="Azimuthal averaged power spectrum density",
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="k",
    yaxis_title="P(k)"
)
fig.show()

## Blur effect (Gaussian)

In [ ]:
from PIL import ImageFilter
blur_image = original_image.filter(ImageFilter.GaussianBlur(5))
blur_numpy_image = convert_numpy_image(blur_image)

fourier_image_blur, kvals_blur, Abins_blur = get_image_power_spectrum(blur_numpy_image)

In [ ]:
plot_images(image1=original_image, title1='Original', image2=blur_image, title2='Blur')

In [ ]:
plot_fourier_image(data1=fourier_image, title1 = "Original", data2=fourier_image_blur, title2="Blur")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=kvals_blur,
    y=Abins_blur,
    mode='lines+markers',
    name='blur'
))
fig.add_trace(go.Scatter(
    x=kvals,
    y=Abins,
    mode='lines+markers',
    name='origianl'
))
  
fig.update_layout(
    title="Azimuthal averaged power spectrum density",
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="k",
    yaxis_title="P(k)"
)
fig.show()

## Contrast

In [ ]:
from PIL import ImageEnhance
contrast_effect = "High contrast"
contrast_image = ImageEnhance.Contrast(original_image).enhance(1.5)
contrast_numpy_image = convert_numpy_image(contrast_image)

fourier_image_contrast, kvals_contrast, Abins_contrast = get_image_power_spectrum(contrast_numpy_image)

In [ ]:
plot_images(image1=original_image, title1='Original', image2=contrast_image, title2=contrast_effect)

In [ ]:
plot_fourier_image(data1=fourier_image, title1 = "Original", data2=fourier_image_contrast, title2=contrast_effect)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=kvals_contrast,
    y=Abins_contrast,
    mode='lines+markers',
    name=contrast_effect
))
  
fig.add_trace(go.Scatter(
    x=kvals,
    y=Abins,
    mode='lines+markers',
    name='origianl'
))
  
fig.update_layout(
    title="Azimuthal averaged power spectrum density",
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="k",
    yaxis_title="P(k)"
)
fig.show()

## Saturation

In [ ]:
from PIL import ImageEnhance
saturation_effect = "High saturation"
saturation_image = ImageEnhance.Color(original_image).enhance(1.5)
saturation_numpy_image = convert_numpy_image(saturation_image)

fourier_image_saturation, kvals_saturation, Abins_saturation = get_image_power_spectrum(saturation_numpy_image)

In [ ]:
plot_images(image1=original_image, title1='Original', image2=saturation_image, title2=saturation_effect)

In [ ]:
plot_fourier_image(data1=fourier_image, title1 = "Original", data2=fourier_image_saturation, title2=saturation_effect)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=kvals_saturation,
    y=Abins_saturation,
    mode='lines+markers',
    name=saturation_effect
))
  
fig.add_trace(go.Scatter(
    x=kvals,
    y=Abins,
    mode='lines+markers',
    name='origianl'
))
  
fig.update_layout(
    title="Azimuthal averaged power spectrum density",
    xaxis_type="log",
    yaxis_type="log",
    xaxis_title="k",
    yaxis_title="P(k)"
)
fig.show()

### 3D Ground Truth Bounding Boxes
In this section we render 3d ground truth bounding boxes on top of the captured images.

In [ ]:
import os
import numpy as np

from ipywidgets import interact
from PIL import Image
from datasetinsights.stats.visualization.plots import plot_bboxes3d
from datasetinsights.datasets.synthetic import read_bounding_box_3d

bounding_box_3d_defintion_id = "0bfbe00d-00fa-4555-88d1-471b58449f5c"
def draw_bounding_boxes3d(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    sensor = box_captures.loc[index, "sensor"]

    if 'camera_intrinsic' in sensor:
        projection = np.array(sensor["camera_intrinsic"])
    else:
        projection = np.array([[1,0,0],[0,1,0],[0,0,1]])

    image = Image.open(filename)
    boxes = read_bounding_box_3d(annotations)
    img_with_boxes = plot_bboxes3d(image, boxes, projection)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    box_captures = cap.filter(def_id=bounding_box_3d_defintion_id)
    interact(draw_bounding_boxes3d, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

## Semantic Segmentation
In this section we render the semantic segmentation images on top of the captured images.

In [ ]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")

## Instance Segmentation
In this section we render the instance segmentation images on top of the captured images. Image IDs are mapped to an RGBA color value, below the image we include a preview of the mapping between colors and IDs.

In [ ]:
def instance_sorter(instance):
    return instance["instance_id"]

def draw_with_instance_segmentation(index, opacity):
    filename = os.path.join(data_root, inst_caps.loc[index, "filename"])
    seg_filename = os.path.join(data_root, inst_caps.loc[index, "annotation.filename"])

    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)

    anns = inst_caps.loc[index, "annotation.values"].copy()
    anns.sort(key=instance_sorter)

    count = min(5, len(anns))
    print("First {} ID entries:".format(count))

    for i in range(count):
        color = anns[i].get("color")
        print ("{} => Color({:>3}, {:>3}, {:>3})".format(anns[i].get("instance_id"), color.get("r"), color.get("g"), color.get("b")))

try:
    inst_seg_def_id = "1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf"
    inst_caps = cap.filter(def_id=inst_seg_def_id)
    interact(draw_with_instance_segmentation, index=(0, inst_caps.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No instance segmentation images found")

## Keypoints
In this section we render the keypoint labeled data for the captured frame.

In [ ]:
from datasetinsights.stats.visualization.plots import plot_keypoints


def draw_human_pose(index):
    filename = os.path.join(data_root, keypoint_caps.loc[index, "filename"])
    annotations = keypoint_caps.loc[index, "annotation.values"]
    templates = ann_def.get_definition(keypoint_def_id)['spec']
    img = Image.open(filename)
    img_with_pose = plot_keypoints(img, annotations, templates)
    img_with_pose.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_pose)


try:
    keypoint_def_id = "8b3ef246-daa7-4dd5-a0e8-a943f6e7f8c2"
    keypoint_caps = cap.filter(def_id=keypoint_def_id)
    interact(draw_human_pose, index=(0, keypoint_caps.shape[0] - 1))
except DefinitionIDError:
    print("No keypoint data found")
